# Thai-English Sentiment Analysis

This notebook creates a sentiment analysis model that supports both Thai and English text. The model classifies text as Positive, Neutral, or Negative.

Created for: ugritchaichana
Date: 2025-04-06

## 1. Setup and Installation

In [22]:
# Install required libraries
!pip install -q pandas numpy scikit-learn pythainlp openpyxl matplotlib seaborn requests beautifulsoup4 tqdm

In [23]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from io import BytesIO
import os
import re
import time
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import pickle
from pythainlp.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('ggplot')
sns.set(font_scale=1.2)

# Check if we have TPU or GPU (though we'll use CPU for most operations)
!nvidia-smi || echo "Running on CPU"

ModuleNotFoundError: No module named 'pythainlp'

## 2. Load Initial Dataset (Your Current Dataset)

In [ ]:
# Load your initial dataset
initial_file_path = "https://drive.google.com/uc?id=1wB6Xl44YnpJDt9YRqaIcAEw_BniL_fel"
print("Downloading initial dataset...")
response = requests.get(initial_file_path)

if response.status_code == 200:
    # Load as Excel
    initial_df = pd.read_excel(BytesIO(response.content), engine="openpyxl")
    print("Initial dataset loaded successfully!")
    display(initial_df.head())

    # Create the standardized format
    initial_dataset = pd.DataFrame({
        'text': initial_df['Statement'],
        'sentiment': initial_df['Sentiment'],
        'source': 'initial_dataset'
    })

    print(f"Loaded {len(initial_dataset)} records from initial dataset")

    # Check sentiment distribution
    print("\nSentiment distribution in initial dataset:")
    print(initial_dataset['sentiment'].value_counts())
else:
    print(f"Error importing initial dataset: {response.status_code}")
    initial_dataset = pd.DataFrame(columns=['text', 'sentiment', 'source'])

## 3. Create Synthetic Sentiment Data

In [ ]:
# Create synthetic data to augment our dataset
def create_synthetic_examples(base_examples, n_variations=3):
    """
    Create synthetic examples by adding variations to existing examples
    """
    if len(base_examples) == 0:
        print("No base examples provided for synthetic data generation.")
        return pd.DataFrame(columns=['text', 'sentiment', 'source'])

    # Define modifiers by sentiment
    thai_pos_modifiers = ['มาก', 'มากๆ', 'สุดๆ', 'ที่สุด', 'จริงๆ', 'เหลือเกิน', 'สุดยอด', 'ดีเยี่ยม']
    thai_neu_modifiers = ['พอใช้', 'ปานกลาง', 'ธรรมดา', 'ทั่วไป', 'พอได้', 'ไม่เลว']
    thai_neg_modifiers = ['แย่', 'แย่มาก', 'ไม่ดี', 'ห่วย', 'แย่สุดๆ', 'เลวร้าย', 'ผิดหวัง']

    eng_pos_modifiers = ['very good', 'excellent', 'fantastic', 'amazing', 'great', 'love it']
    eng_neu_modifiers = ['okay', 'fine', 'standard', 'acceptable', 'average', 'not bad']
    eng_neg_modifiers = ['bad', 'terrible', 'awful', 'horrible', 'disappointing', 'poor']

    # Create new examples
    new_examples = []

    for _, row in base_examples.iterrows():
        text = row['text']
        sentiment = row['sentiment']

        # Create variations
        for _ in range(n_variations):
            # Choose appropriate modifiers based on sentiment
            if sentiment == 'Positive':
                thai_mod = np.random.choice(thai_pos_modifiers)
                eng_mod = np.random.choice(eng_pos_modifiers)
            elif sentiment == 'Neutral':
                thai_mod = np.random.choice(thai_neu_modifiers)
                eng_mod = np.random.choice(eng_neu_modifiers)
            else:  # Negative
                thai_mod = np.random.choice(thai_neg_modifiers)
                eng_mod = np.random.choice(eng_neg_modifiers)

            # Create new text by appending modifiers
            # 50% chance for Thai modifier, 50% for English
            if np.random.random() > 0.5:
                new_text = f"{text} {thai_mod}"
            else:
                new_text = f"{text} {eng_mod}"

            new_examples.append({
                'text': new_text,
                'sentiment': sentiment,
                'source': 'synthetic'
            })

    return pd.DataFrame(new_examples)

# Generate synthetic examples based on initial dataset
synthetic_df = create_synthetic_examples(initial_dataset, n_variations=5)

print(f"Created {len(synthetic_df)} synthetic examples")
print("\nSentiment distribution in synthetic dataset:")
print(synthetic_df['sentiment'].value_counts())
display(synthetic_df.head(3))

## 4. Create English Sentiment Data

In [ ]:
# Create English sentiment examples
def create_english_examples():
    """
    Create a small dataset of English sentiment examples
    """
    # Define examples for each sentiment category
    positive_examples = [
        "This is excellent!",
        "I love this product.",
        "Great service and fast delivery.",
        "The quality is amazing.",
        "Very satisfied with my purchase.",
        "Fantastic experience overall.",
        "The staff was very helpful.",
        "I would definitely recommend this.",
        "Exceeded my expectations.",
        "Best product I've ever bought."
    ]

    neutral_examples = [
        "It's okay.",
        "Average quality for the price.",
        "Neither good nor bad.",
        "It works as expected.",
        "Standard product, nothing special.",
        "Acceptable but could be better.",
        "Does the job, but that's about it.",
        "Reasonable for the price.",
        "It's fine I guess.",
        "Not bad, not great either."
    ]

    negative_examples = [
        "Very disappointed with this.",
        "Poor quality product.",
        "Would not recommend at all.",
        "Terrible customer service.",
        "Doesn't work as advertised.",
        "Complete waste of money.",
        "The worst experience I've had.",
        "Very frustrated with this purchase.",
        "Extremely poor quality control.",
        "I regret buying this."
    ]

    # Create dataframe
    data = []

    # Add positive examples
    for text in positive_examples:
        data.append({'text': text, 'sentiment': 'Positive', 'source': 'english_examples'})

    # Add neutral examples
    for text in neutral_examples:
        data.append({'text': text, 'sentiment': 'Neutral', 'source': 'english_examples'})

    # Add negative examples
    for text in negative_examples:
        data.append({'text': text, 'sentiment': 'Negative', 'source': 'english_examples'})

    return pd.DataFrame(data)

# Create English examples
english_df = create_english_examples()

print(f"Created {len(english_df)} English examples")
print("\nSentiment distribution in English dataset:")
print(english_df['sentiment'].value_counts())
display(english_df.head(3))

## 5. Create Mixed Thai-English Examples

In [ ]:
def create_mixed_examples():
    """
    Create examples with mixed Thai and English text
    """
    # Thai phrases by sentiment
    thai_positive = [
        "สินค้าดี", "คุณภาพดี", "บริการดีมาก", "ชอบมาก", "ประทับใจ",
        "คุ้มค่าเงิน", "รู้สึกดี", "พอใจมาก", "เยี่ยมมาก", "ดีเลิศ"
    ]

    thai_neutral = [
        "พอใช้ได้", "ปานกลาง", "ธรรมดา", "ก็โอเค", "เฉยๆ",
        "ไม่เลวนะ", "ทั่วไป", "พอทำเนา", "ไม่แย่", "พอได้"
    ]

    thai_negative = [
        "ไม่ดีเลย", "แย่มาก", "ผิดหวัง", "ไม่พอใจ", "คุณภาพต่ำ",
        "บริการแย่", "เสียดายเงิน", "ไม่ประทับใจ", "ราคาแพงเกินไป", "ใช้งานยาก"
    ]

    # English phrases by sentiment
    eng_positive = [
        "great product", "excellent service", "love it", "highly recommend", "fantastic",
        "amazing experience", "very satisfied", "worth the money", "impressive", "top quality"
    ]

    eng_neutral = [
        "it's okay", "average", "standard quality", "not bad", "as expected",
        "mediocre", "acceptable", "fair enough", "moderate", "reasonable"
    ]

    eng_negative = [
        "disappointing", "poor quality", "would not recommend", "terrible", "waste of money",
        "very frustrating", "doesn't work well", "not worth it", "bad experience", "regret buying"
    ]

    # Create mixed examples
    mixed_data = []

    # Thai first, then English
    for thai in thai_positive:
        for eng in eng_positive[:3]:  # Limit combinations to avoid too many examples
            mixed_data.append({
                'text': f"{thai} {eng}",
                'sentiment': 'Positive',
                'source': 'mixed_examples'
            })

    for thai in thai_neutral:
        for eng in eng_neutral[:3]:
            mixed_data.append({
                'text': f"{thai} {eng}",
                'sentiment': 'Neutral',
                'source': 'mixed_examples'
            })

    for thai in thai_negative:
        for eng in eng_negative[:3]:
            mixed_data.append({
                'text': f"{thai} {eng}",
                'sentiment': 'Negative',
                'source': 'mixed_examples'
            })

    # English first, then Thai
    for eng in eng_positive:
        for thai in thai_positive[:3]:
            mixed_data.append({
                'text': f"{eng} {thai}",
                'sentiment': 'Positive',
                'source': 'mixed_examples'
            })

    for eng in eng_neutral:
        for thai in thai_neutral[:3]:
            mixed_data.append({
                'text': f"{eng} {thai}",
                'sentiment': 'Neutral',
                'source': 'mixed_examples'
            })

    for eng in eng_negative:
        for thai in thai_negative[:3]:
            mixed_data.append({
                'text': f"{eng} {thai}",
                'sentiment': 'Negative',
                'source': 'mixed_examples'
            })

    # Take a sample to avoid too many examples
    if len(mixed_data) > 300:
        import random
        random.shuffle(mixed_data)
        mixed_data = mixed_data[:300]

    return pd.DataFrame(mixed_data)

# Create mixed examples
mixed_df = create_mixed_examples()

print(f"Created {len(mixed_df)} mixed Thai-English examples")
print("\nSentiment distribution in mixed dataset:")
print(mixed_df['sentiment'].value_counts())
display(mixed_df.head(3))

## 6. Try to Get Wisesight Dataset (Optional)

If this fails, the notebook will continue with other data.

In [ ]:
try:
    !git clone https://github.com/PyThaiNLP/wisesight-sentiment.git

    # Load the datasets
    wisesight_data = []
    for split in ['train', 'valid']:
        try:
            file_path = f'wisesight-sentiment/corpus/wisesight_sentiment_{split}.txt'

            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()

                for i in range(0, len(lines), 2):
                    if i+1 < len(lines):
                        sentiment = lines[i].strip()
                        text = lines[i+1].strip()

                        # Map sentiment
                        if sentiment == 'pos':
                            sentiment = 'Positive'
                        elif sentiment == 'neu':
                            sentiment = 'Neutral'
                        elif sentiment == 'neg':
                            sentiment = 'Negative'
                        else:
                            continue  # Skip questions or other

                        wisesight_data.append({
                            'text': text,
                            'sentiment': sentiment,
                            'source': f'wisesight_{split}'
                        })
            else:
                print(f"File not found: {file_path}")

        except Exception as e:
            print(f"Error processing Wisesight {split} set: {e}")

    # Create dataframe
    if wisesight_data:
        wisesight_df = pd.DataFrame(wisesight_data)
        # Take a sample to keep dataset balanced
        wisesight_df = wisesight_df.sample(min(1000, len(wisesight_df)), random_state=42)
        print(f"Loaded {len(wisesight_df)} records from Wisesight dataset")
        print("\nSentiment distribution in Wisesight dataset:")
        print(wisesight_df['sentiment'].value_counts())
    else:
        print("No data loaded from Wisesight")
        wisesight_df = pd.DataFrame(columns=['text', 'sentiment', 'source'])

except Exception as e:
    print(f"Error loading Wisesight dataset: {e}")
    wisesight_df = pd.DataFrame(columns=['text', 'sentiment', 'source'])

## 7. Combine All Datasets

In [ ]:
# Prepare datasets with standardized columns
datasets = [
    initial_dataset[['text', 'sentiment', 'source']],
    synthetic_df[['text', 'sentiment', 'source']],
    english_df[['text', 'sentiment', 'source']],
    mixed_df[['text', 'sentiment', 'source']],
]

# Add Wisesight if available
if 'wisesight_df' in locals() and len(wisesight_df) > 0:
    datasets.append(wisesight_df[['text', 'sentiment', 'source']])

# Combine all datasets
combined_df = pd.concat(datasets, ignore_index=True)

# Remove any rows with missing values
combined_df = combined_df.dropna()

# Basic cleaning
combined_df['text'] = combined_df['text'].astype(str).apply(lambda x: x.strip())

# Remove duplicate texts
combined_df = combined_df.drop_duplicates(subset=['text'])

print(f"Combined dataset: {len(combined_df)} records")
print("\nSentiment distribution in combined dataset:")
print(combined_df['sentiment'].value_counts())
print("\nSource distribution:")
print(combined_df['source'].value_counts())

# Visualize sentiment distribution
plt.figure(figsize=(10, 6))
sns.countplot(y='sentiment', data=combined_df, palette='viridis')
plt.title('Sentiment Distribution in Combined Dataset')
plt.tight_layout()
plt.show()

## 8. Text Preprocessing

In [ ]:
def clean_text(text):
    """
    Clean text data by removing URLs, user mentions, extra spaces, etc.
    """
    if not isinstance(text, str):
        return ""

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove user mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply cleaning
combined_df['cleaned_text'] = combined_df['text'].apply(clean_text)

# Custom tokenizer for Thai + English
def tokenize_text(text):
    """
    Tokenize text using pythainlp for Thai language support
    """
    if not isinstance(text, str):
        return []

    # Tokenize with pythainlp's newmm tokenizer
    try:
        tokens = word_tokenize(text, engine='newmm')
        # Remove tokens that are just whitespace or single characters
        tokens = [token for token in tokens if token.strip() and len(token.strip()) > 1]
        return tokens
    except Exception as e:
        print(f"Error tokenizing text: {text}, Error: {e}")
        # Fall back to basic split
        return [t for t in text.split() if len(t.strip()) > 1]

# Test the tokenizer on a mixed language example
test_text = "สินค้าดีมาก I really like this product นี่เป็นการซื้อครั้งที่ 2 แล้ว will buy again"
print(f"Original: {test_text}")
print(f"Tokenized: {tokenize_text(test_text)}")

## 9. Model Training

In [ ]:
# Prepare data for modeling
X = combined_df['cleaned_text']
y = combined_df['sentiment']

# Split into training and testing sets (stratified by sentiment)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]} examples")
print(f"Testing set size: {X_test.shape[0]} examples")

In [ ]:
# Create a pipeline with TF-IDF and classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        tokenizer=tokenize_text,
        ngram_range=(1, 2),
        max_features=30000,
        min_df=2,
        sublinear_tf=True  # Apply sublinear tf scaling (log)
    )),
    ('classifier', LogisticRegression(
        C=1.0,
        max_iter=1000,
        class_weight='balanced',
        solver='liblinear',
        random_state=42
    ))
])

# Train the model
print("Training the model...")
%time pipeline.fit(X_train, y_train)
print("Model training complete!")

## 10. Model Evaluation

In [ ]:
# Evaluate on test set
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=pipeline.classes_,
           yticklabels=pipeline.classes_)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

In [ ]:
# Evaluate on different data sources
source_results = {}

# Test on each source
for source in combined_df['source'].unique():
    source_data = combined_df[combined_df['source'] == source]
    if len(source_data) > 10:  # Only test if we have enough examples
        X_source = source_data['cleaned_text']
        y_source = source_data['sentiment']

        # Predict
        y_source_pred = pipeline.predict(X_source)
        source_accuracy = accuracy_score(y_source, y_source_pred)
        source_results[source] = source_accuracy

# Display results by source
print("Accuracy by data source:")
for source, acc in source_results.items():
    print(f"{source}: {acc:.4f}")

# Visualize
plt.figure(figsize=(10, 6))
sns.barplot(x=list(source_results.keys()), y=list(source_results.values()))
plt.title('Model Accuracy by Data Source')
plt.ylabel('Accuracy')
plt.xlabel('Data Source')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 11. Save the Model

In [ ]:
# Save the trained model
model_filename = 'thai_english_sentiment_model.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(pipeline, f)
print(f"Model saved as '{model_filename}'")

# Save the combined dataset for future use
combined_df.to_csv('thai_english_sentiment_dataset.csv', index=False)
print("Dataset saved as 'thai_english_sentiment_dataset.csv'")

# Download files to your computer
from google.colab import files
files.download(model_filename)
files.download('thai_english_sentiment_dataset.csv')

## 12. Test with Examples

In [ ]:
def predict_sentiment(text, model=pipeline):
    """
    Predict sentiment for text in Thai, English, or mixed.

    Args:
        text (str): Input text
        model: Trained model pipeline

    Returns:
        tuple: (predicted sentiment, confidence percentage, all probabilities)
    """
    # Clean the text
    cleaned_text = clean_text(text)

    # Make prediction
    prediction = model.predict([cleaned_text])[0]

    # Get prediction probabilities
    proba = model.predict_proba([cleaned_text])[0]
    confidence = max(proba) * 100

    # Get all class probabilities with labels
    all_probas = {cls: prob*100 for cls, prob in zip(model.classes_, proba)}

    return prediction, confidence, all_probas

# Test with examples
test_examples = [
    # Thai examples
    "สินค้าดีมากๆ คุณภาพเยี่ยม",                        # Very good product, excellent quality
    "พอใช้ได้ ไม่ดีมาก ไม่แย่มาก",                      # It's okay, not very good, not very bad
    "แย่มาก ผิดหวังมากกับสินค้าตัวนี้",                 # Very bad, very disappointed with this product
    # English examples
    "This product is amazing, exceeded my expectations!",
    "It's an average product, does what it's supposed to do.",
    "Terrible service, would not recommend to anyone.",
    # Mixed Thai-English examples
    "ชอบสินค้านี้มาก Best purchase ever!",             # Love this product, best purchase ever
    "Product is okay, แต่ราคาแพงไปนิด",                # Product is okay, but a bit expensive
    "Disappointed with the quality ใช้แล้วพังเลย"      # Disappointed with the quality, it broke after using
]

# Test and display results
results = []
for text in test_examples:
    sentiment, confidence, all_probas = predict_sentiment(text)
    results.append({
        'text': text,
        'predicted_sentiment': sentiment,
        'confidence': f"{confidence:.2f}%",
        'all_probabilities': all_probas
    })

# Display results in a formatted way
for i, result in enumerate(results):
    print(f"\nExample {i+1}:")
    print(f"Text: {result['text']}")
    print(f"Predicted Sentiment: {result['predicted_sentiment']}")
    print(f"Confidence: {result['confidence']}")
    print("Class Probabilities:")
    for cls, prob in result['all_probabilities'].items():
        print(f"  - {cls}: {prob:.2f}%")

## 13. Interactive Testing Tool

In [ ]:
def analyze_text():
    text = input("Enter text (Thai/English/Mixed) to analyze (or 'q' to quit): ")
    if text.lower() == 'q':
        return False

    if text.strip() == '':
        print("Please enter some text to analyze.")
        return True

    sentiment, confidence, all_probas = predict_sentiment(text)

    print(f"\nText: {text}")
    print(f"Predicted Sentiment: {sentiment}")
    print(f"Confidence: {confidence:.2f}%")
    print("Class Probabilities:")
    for cls, prob in all_probas.items():
        print(f"  - {cls}: {prob:.2f}%")
    print("\n" + "-"*50)
    return True

print("=== Thai-English Sentiment Analysis Tool ===\n")
print("Examples you can try:")
print("1. สินค้าดีมากๆ คุณภาพเยี่ยม (Very good product, excellent quality)")
print("2. Product is okay, แต่ราคาแพงไปนิด (Product is okay, but a bit expensive)")
print("3. Terrible service, never again!")
print("\nType 'q' to quit when you're done.\n")
print("-"*50)

continue_analysis = True
while continue_analysis:
    continue_analysis = analyze_text()

## 14. Next Steps: How to Expand Your Dataset

### เพิ่มข้อมูลในชุดข้อมูลของคุณ

1. **Web Scraping** - เก็บข้อมูลจากเว็บไซต์รีวิวภาษาไทย เช่น:
   - Pantip
   - Wongnai
   - Twitter (ใช้ Twitter API)
   - Facebook Comments (ใช้ Facebook Graph API)

2. **ใช้ Dataset สาธารณะ** - ชุดข้อมูลเพิ่มเติมที่สามารถดาวน์โหลดได้:
   - [VISTEC-TP-TH-Twitter-Sentiment](https://github.com/mrpeerat/VISTEC-TP-TH-Twitter-Sentiment) - ชุดข้อมูลความรู้สึกภาษาไทยบน Twitter
   - [Thai-Social-Media-Crisis-Dataset](https://github.com/jitkapat/Thai-Social-Media-Crisis-Dataset) - ชุดข้อมูลวิกฤตในสื่อสังคมไทย
   - [Thai Review and Rating Dataset](https://github.com/PyThaiNLP/thai-review-and-rating-dataset) - ชุดข้อมูลรีวิวและคะแนนภาษาไทย

3. **สร้างข้อมูลด้วย AI** - ใช้ AI สร้างข้อมูลเพิ่มเติม:
   - ใช้ API ของ ChatGPT หรือ GPT-4 เพื่อสร้างข้อความตัวอย่างมากขึ้น
   - ใช้การแปลภาษาเพื่อแปลชุดข้อมูลภาษาอังกฤษที่มีอยู่เป็นภาษาไทย

4. **Data Augmentation** - เทคนิคการเพิ่มข้อมูล:
   - แทนที่คำด้วยคำที่มีความหมายเหมือนกัน
   - สลับตำแหน่งประโยค
   - เพิ่มหรือลบคำที่ไม่สำคัญ

### พัฒนาโมเดลให้ดีขึ้น

1. **ใช้ Deep Learning** - โมเดลที่ซับซ้อนมากขึ้น:
   - [wangchanberta](https://huggingface.co/airesearch/wangchanberta-base-att-spm-uncased) - BERT สำหรับภาษาไทย
   - [XLM-RoBERTa](https://huggingface.co/xlm-roberta-base) - สนับสนุนหลายภาษา
   - [mBERT](https://huggingface.co/bert-base-multilingual-cased) - BERT หลายภาษา

2. **การใช้ Transformer Models** - ตัวอย่างโค้ด:
```python
# ติดตั้งไลบรารี
!pip install transformers torch

# โค้ดตัวอย่างการใช้ wangchanberta
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# โหลดโมเดลและ tokenizer
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
model = AutoModelForSequenceClassification.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", num_labels=3)

# ฟังก์ชันทำนาย
def predict_with_transformer(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probabilities
```

### แหล่งข้อมูลเพิ่มเติม

1. [PyThaiNLP](https://github.com/PyThaiNLP/pythainlp) - ไลบรารีประมวลผลภาษาธรรมชาติสำหรับภาษาไทย
2. [Thai NLP Resource](https://github.com/kobkrit/thai-nlp-resource) - รวมลิงก์ข้อมูลและเครื่องมือสำหรับ NLP ภาษาไทย
3. [VISTEC-AI Research](https://github.com/vistec-AI) - งานวิจัย AI และ NLP สำหรับภาษาไทย